## Importing Relevant Libraries

In [ ]:
import os  
from openai import AzureOpenAI
import json
from dotenv import load_dotenv

load_dotenv()

## Setting Up the ENV Variables

In [ ]:

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("OPENAI_API_HOST")
)
print(os.getenv("OPENAI_API_KEY"))

## Calling Chat Completition With User Query

In [ ]:
result = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
       {"role": "user", "content": "Hello, how are you?"}
    ]) 

print(result)

## Calling Chat Completition With System + User Query

In [ ]:
result = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content": "You are a helpful assistant who replies in the style of dolly parton."},
        {"role": "user", "content": "What is the purpose of life?"},
    ]) 

print(result)

In [ ]:
result.id

## Calling OpenAI Embeddings

In [ ]:
response = client.embeddings.create(  
    input="Hello", model="text-embedding-ada-002"
)

print(response.data[0].embedding)

## LogProbs and Top_logprobs

In [ ]:
result = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
       {"role": "user", "content": "Hello, how are you?"}
    ], logprobs=True, top_logprobs=3) 

print(result)

In [ ]:
print(result.choices[0].logprobs) 

In [ ]:
import math
math.exp(-0.00026169344)

## Function Calling

In [ ]:
function_schema = [  
    {
        "type": "function",
        "function": {
            "name": "calculate_circle_area",  
            "description": "Calculate the area of a circle given the radius.",
            "parameters": {  
                "type": "object",  
                "properties": {  
                    "radius": {  
                        "type": "number",  
                        "description": "The radius of the circle in centimeters."  
                    }  
                },  
                "required": ["radius"]  
            }  
        }
    }  
]  

In [ ]:
def calculate_circle_area(radius):  
    return math.pi * radius * radius

In [ ]:
user_message = "Calculate the area of a circle with a radius of 5 cm."

In [ ]:
response = client.chat.completions.create(  
    model="gpt-4o-mini",
    messages=[  
        {"role": "user", "content": user_message}  
    ],  
    tools=function_schema,  
    tool_choice="auto"  
)

print(response)

In [ ]:
tool_call = response.choices[0].message.tool_calls[0]
print(tool_call)

In [ ]:
tool_call.function.name

In [ ]:
tool_call.function.arguments

In [ ]:
calculate_circle_area(5)

In [ ]:
message = response.choices[0].message
message.tool_calls
arguments = json.loads(message.tool_calls[0].function.arguments)  
arguments
arguments.get("radius")

In [ ]:
tool_call = response.choices[0].message.tool_calls[0]
tool_call

In [ ]:
if message.tool_calls:  
    function_name = message.tool_calls[0].function.name
    arguments = json.loads(message.tool_calls[0].function.arguments)   
      
    if function_name == "calculate_circle_area":  
        radius = arguments.get("radius")  
        area = calculate_circle_area(radius)  
          
        # Convert the result to a string to send back to the model  
        function_response = str(area)  
          
        # Send the result back to the model  
        second_response = client.chat.completions.create(  
            model="gpt-4o-mini",  
            messages=[  
                {"role": "user", "content": user_message},  
                message,  
                {  
                    "role": "tool",  
                    "tool_call_id": tool_call.id,  
                    "content": function_response  
                }  
            ]  
        )  
          
        final_answer = second_response.choices[0].message.content
        print('final_answer:', final_answer)  
else:  
    # If no function call is made, just print the assistant's response  
    print(message['content'])  

In [ ]:
print(second_response)